In [152]:
import torch
import numpy as np
from utils import batched_eye_like, skew_symmetric, to_homogeneous, from_homogeneous

In [155]:
points = torch.randn((50,2))

In [154]:
R = torch.eye(3).to(points)
t = points.new_tensor([1,1,0]).float()

In [156]:
z0 = torch.ones_like(points[...,0])*5

In [157]:
if isinstance(points, torch.Tensor):
    pad = points.new_ones(points.shape[:-1]+(1,))
    points_h = torch.cat([points, pad], dim=-1)

In [160]:
p_3d = points_h*z0[...,None]

In [161]:
p_3d_1 = p_3d@R + t

In [162]:
p_proj_1 = from_homogeneous(p_3d_1)

In [165]:
J_p_T = torch.cat([batched_eye_like(p_3d_1, 3), -skew_symmetric(p_3d_1)], -1)
print("J_p_T shape is ", J_p_T.shape)

J_p_T shape is  torch.Size([50, 3, 6])


In [166]:
shape = p_3d_1.shape[:-1]

In [167]:
o, z = p_3d_1.new_ones(shape), p_3d_1.new_zeros(shape)

In [168]:
J_e_p =torch.stack([o, z, -p_3d_1[...,0]/p_3d_1[...,2], z, o, -p_3d_1[...,1]/p_3d_1[...,2]], dim = -1)

In [169]:
J_e_p = J_e_p.reshape(shape + (2,3))/p_3d_1[...,2,None, None]
print("J_e_p shape is ", J_e_p.shape)

J_e_p shape is  torch.Size([50, 2, 3])


In [171]:
J_f_p = torch.randn((50, 10, 2))
print("J_f_p shape is ", J_f_p.shape)

J_f_p shape is  torch.Size([50, 10, 2])


In [173]:
J_e_T = J_f_p @ J_e_p @ J_p_T
print("J_e_T shape is ", J_e_T.shape)

J_e_T shape is  torch.Size([50, 10, 6])


In [174]:
error = torch.ones((50,10))

In [177]:
Grad = torch.einsum('...ijk,...ij->...ik', J_e_T, error)
print("Grad shape is ", Grad.shape)

Grad shape is  torch.Size([50, 6])


In [100]:
J_e_T.shape, error.shape

(torch.Size([50, 2, 6]), torch.Size([50, 2]))

In [102]:
import kornia

In [115]:
t = torch.randn((1,3,4,4))

In [116]:
grad = kornia.filters.SpatialGradient()(t)

In [122]:
f = grad[:,:,0,:,:]

In [123]:
g = f.reshape((3,4,4))

In [126]:
a = torch.randn((3,24,24))

In [139]:
ids = torch.tensor([[1,2],[3,4],[2,5]]).t()

In [140]:
ids

tensor([[1, 3, 2],
        [2, 4, 5]])

In [141]:
a[:,ids].size()

torch.Size([3, 2, 3, 24])

In [145]:
a.shape

torch.Size([3, 24, 24])

In [146]:
x = torch.randn((3,4))
y = torch.randn((3,4))

In [151]:
torch.cat((x[...,None], y[...,None]), -1).shape

torch.Size([3, 4, 2])